In [1]:
import sys
import os

# Get the parent directory of the notebook (project root)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from dotenv import load_dotenv

load_dotenv()

In [2]:
import os

if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0  # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os

    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
tf.get_logger().setLevel("ERROR")

# Set random seed for reproducibility
sionna.config.seed = 42

In [3]:
import time

import numpy as np
import torch
from sionna.channel.tr38901 import CDL
from sionna.utils import ebnodb2no
from tqdm import tqdm

from src.channels.cdl_channel import (
    cdl,
    sampling_channel_freq,
    sampling_channel_time,
    no,
)
from src.channels.channel_est.ls_channel import ChannelEstimator
from src.channels.channel_est.ml_channel import VAE
from src.channels.lmmse_equalizer import lmmse_equalizer
from src.data.binary_sources import binary_sources
from src.data.qam_demapper import qam_demapper
from src.data.qam_mapper import qam_mapper
from src.data.response import response_freqency_domain, response_time_domain
from src.evals.ber import ber
from src.ldpc.ldpc_decoder import ldpc_decoder
from src.ldpc.ldpc_encoder import ldpc_encoder
from src.ml.gen_data import get_pilot_matrix, remove_nulls_subcarriers
from src.ml.transform import MinMaxScaler4D
from src.ml.utils import (
    compute_mean_std,
    nmse_func,
    reshape_data,
    reversed_reshape_data,
    replicate_to_shape,
    ssim_func,
)
from src.ofdm.ofdm_demodulation import ofdm_demodulation
from src.ofdm.ofdm_modulation import ofdm_modulation
from src.ofdm.ofdm_resource_grids import resource_grid_mapper, rg
from src.settings.antenna import bs_array, ut_array
from src.settings.config import (
    batch_size,
    bits_per_symbol,
    carrier_frequency,
    cdl_model,
    code_rate,
    delay_spread,
    direction,
    ebno_db,
    num_bs_ant,
    num_effective_subcarriers,
    num_ofdm_symbols,
    num_rx,
    num_streams_per_tx,
    num_tx,
    num_ut_ant,
    number_of_bits,
    speed,
)
from src.settings.ml import device, number_of_samples
from src.utils.plots import plot_channel_frequency_domain, plot_symbols

scaler = MinMaxScaler4D()
h_freq_shape = remove_nulls_subcarriers(sampling_channel_freq()).shape
root_dir = ""

In [4]:
def set_up_model():
    src_dir = f"txant_{num_ut_ant}_rxant_{num_bs_ant}_speed_{speed}_samples_{number_of_samples}_ebno_0"
    h_freqs = np.load(f"{root_dir}/data/{src_dir}/h_freqs.npy")

    h_freqs = reshape_data(h_freqs)

    h_freqs = scaler.fit_transform(h_freqs)

    mu_h, std_h = compute_mean_std(h_freqs)
    mu_h = mu_h.to(device)
    std_h = std_h.to(device)

    vae = VAE(
        h_freqs.shape[1:],
        mu_h,
        std_h,
    )

    vae.load_state_dict(
        torch.load(
            f"{root_dir}/results/checkpoints/{src_dir}/vae_fold_2_best.pth",
            map_location=device,
        )["state_dict"]
    )
    vae = vae.to(device)
    vae.eval()

    return vae


vae = set_up_model()

In [6]:
def nmse_channel_estimation_ai(number_of_trials, no=no, cdl=cdl):
    nmse_scores = []
    for _ in tqdm(range(number_of_trials)):
        binary_values = binary_sources(
            [batch_size, num_tx, num_streams_per_tx, number_of_bits]
        )
        encoded_binary_values = ldpc_encoder(binary_values)
        qam_symbols = qam_mapper(encoded_binary_values)
        mapped_qam_symbol = resource_grid_mapper(qam_symbols)
        h_freq = sampling_channel_freq(cdl=cdl)

        # response signal on frequency domain via CDL channel model + AWGN noise
        response_symbols = response_freqency_domain(mapped_qam_symbol, h_freq, no=no)

        pilot_matrix = get_pilot_matrix(response_symbols)
        # pilot_matrix = replicate_to_shape(pilot_matrix, h_freq_shape)
        pilot_matrix = reshape_data(pilot_matrix)
        pilot_matrix = scaler.fit_transform(pilot_matrix, False).to(device)
        h_est_hat, *values = vae(pilot_matrix)
        h_est_hat = scaler.inverse_transform(h_est_hat.cpu().detach())

        h_est = remove_nulls_subcarriers(h_freq)
        h_est = reshape_data(h_est.numpy())

        nmse_score = nmse_func(
            torch.from_numpy(h_est),
            torch.from_numpy(h_est_hat),
        )
        nmse_scores.append(nmse_score)

    return np.mean(nmse_scores)


def nmse_channel_estimation_ls(
    number_of_trials, inter: str = "nn", cdl=cdl, no=no, order="t-f"
):
    nmse_scores = []
    for _ in tqdm(range(number_of_trials)):
        binary_values = binary_sources(
            [batch_size, num_tx, num_streams_per_tx, number_of_bits]
        )
        encoded_binary_values = ldpc_encoder(binary_values)
        qam_symbols = qam_mapper(encoded_binary_values)
        mapped_qam_symbol = resource_grid_mapper(qam_symbols)
        h_freq = sampling_channel_freq(cdl=cdl)

        # response signal on frequency domain via CDL channel model + AWGN noise
        response_symbols = response_freqency_domain(mapped_qam_symbol, h_freq, no=no)

        channel_estimator = ChannelEstimator(interpolation_factor=inter, order=order)
        h_est_hat, err_var = channel_estimator.estimate(response_symbols)

        h_est_hat = reshape_data(h_est_hat.numpy())

        h_est = remove_nulls_subcarriers(h_freq)
        h_est = reshape_data(h_est.numpy())

        nmse_score = nmse_func(
            torch.from_numpy(h_est_hat),
            torch.from_numpy(h_est),
        )
        nmse_scores.append(nmse_score)

    return np.mean(nmse_scores)

In [7]:
start = time.time()
nmse_score = nmse_channel_estimation_ai(1)
end = time.time()
print(f"Time taken for AI: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00,  4.23it/s]

Time taken for AI: 0.2380504608154297 seconds


In [8]:
start = time.time()
nmse_score = nmse_channel_estimation_ls(1, inter="nn")
end = time.time()
print(f"Time taken for LS+NN: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00, 16.13it/s]

Time taken for LS+NN: 0.06303930282592773 seconds


In [9]:
start = time.time()
nmse_score = nmse_channel_estimation_ls(1, inter="lin")
end = time.time()
print(f"Time taken for LS+Lin: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00, 16.75it/s]

Time taken for LS+Lin: 0.060755014419555664 seconds


In [10]:
start = time.time()
nmse_score = nmse_channel_estimation_ls(1, inter="lmmse", order="s-t-f")
end = time.time()
print(f"Time taken for LS+LMMSE: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00,  2.38it/s]

Time taken for LS+LMMSE: 0.42087650299072266 seconds


In [11]:
import time
import numpy as np

hyperparameters = {
    "ebno_dbs": np.arange(-20, 21, 5),
    "speeds": np.arange(0, 11, 5),
}

number_of_trials = 200
nmse_scores = {}
inference_times = {}  # Dictionary to store inference times

for speed in hyperparameters["speeds"]:
    cdl = CDL(
        cdl_model,
        delay_spread,
        carrier_frequency,
        ut_array,
        bs_array,
        direction,
        min_speed=speed,
    )
    for ebno_db in hyperparameters["ebno_dbs"]:
        nmse_score = {}
        inference_time = {}  # Store times for each model
        no = ebnodb2no(ebno_db, bits_per_symbol, code_rate, rg)

        # Measure inference time for VAE
        start_time = time.time()
        nmse_score["vae"] = nmse_channel_estimation_ai(number_of_trials, no=no, cdl=cdl)
        inference_time["vae"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+NN
        start_time = time.time()
        nmse_score["ls+nn"] = nmse_channel_estimation_ls(
            number_of_trials, inter="nn", cdl=cdl, no=no
        )
        inference_time["ls+nn"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LIN
        start_time = time.time()
        nmse_score["ls+lin"] = nmse_channel_estimation_ls(
            number_of_trials, inter="lin", cdl=cdl, no=no
        )
        inference_time["ls+lin"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        nmse_score["ls+lmmse: t-f"] = nmse_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="t-f"
        )
        inference_time["ls+lmmse: t-f"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        nmse_score["ls+lmmse: t-f-s"] = nmse_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="t-f-s"
        )
        inference_time["ls+lmmse: t-f-s"] = (
            time.time() - start_time
        ) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        nmse_score["ls+lmmse: s-t-f"] = nmse_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="s-t-f"
        )
        inference_time["ls+lmmse: s-t-f"] = (
            time.time() - start_time
        ) / number_of_trials

        nmse_scores[f"speed_{speed}_ebno_db_{ebno_db}"] = nmse_score
        inference_times[f"speed_{speed}_ebno_db_{ebno_db}"] = inference_time

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:09<00:00, 21.03it/s]


In [12]:
import json
import os

save_dir = f"{root_dir}/results/eval/nmse/nmse.json"
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

# Key for the current data
key = f"txant_{num_ut_ant}_rxant_{num_bs_ant}"

# Read existing data if the file exists
existing_data = {}
if os.path.exists(save_dir):
    try:
        with open(save_dir, "r") as f:
            existing_data = json.load(f)
    except json.JSONDecodeError:
        # Handle empty or corrupted JSON file
        existing_data = {}

# Update or append the new data
existing_data[key] = nmse_scores

# Write back to the file
with open(save_dir, "w") as f:
    json.dump(existing_data, f, indent=4)

In [13]:
import json
import os

save_dir = f"{root_dir}/results/eval/nmse/time.json"
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

# Key for the current data
key = f"txant_{num_ut_ant}_rxant_{num_bs_ant}"

# Read existing data if the file exists
existing_data = {}
if os.path.exists(save_dir):
    try:
        with open(save_dir, "r") as f:
            existing_data = json.load(f)
    except json.JSONDecodeError:
        # Handle empty or corrupted JSON file
        existing_data = {}

# Update or append the new data
existing_data[key] = inference_times

# Write back to the file
with open(save_dir, "w") as f:
    json.dump(existing_data, f, indent=4)

# SSIM

In [14]:
def ssim_channel_estimation_ai(number_of_trials, no=no, cdl=cdl):
    ssim_scores = []
    for _ in tqdm(range(number_of_trials)):
        binary_values = binary_sources(
            [batch_size, num_tx, num_streams_per_tx, number_of_bits]
        )
        encoded_binary_values = ldpc_encoder(binary_values)
        qam_symbols = qam_mapper(encoded_binary_values)
        mapped_qam_symbol = resource_grid_mapper(qam_symbols)
        h_freq = sampling_channel_freq(cdl=cdl)

        # response signal on frequency domain via CDL channel model + AWGN noise
        response_symbols = response_freqency_domain(mapped_qam_symbol, h_freq, no=no)

        pilot_matrix = get_pilot_matrix(response_symbols)
        # pilot_matrix = replicate_to_shape(pilot_matrix, h_freq_shape)
        pilot_matrix = reshape_data(pilot_matrix)
        pilot_matrix = scaler.fit_transform(pilot_matrix, False).to(device)
        h_est_hat, *values = vae(pilot_matrix)
        h_est_hat = scaler.inverse_transform(h_est_hat.cpu().detach())

        h_est = remove_nulls_subcarriers(h_freq)
        h_est = reshape_data(h_est.numpy())

        ssim_score = ssim_func(
            torch.from_numpy(h_est),
            torch.from_numpy(h_est_hat),
        )
        ssim_scores.append(ssim_score)

    return np.mean(ssim_scores)


def ssim_channel_estimation_ls(
    number_of_trials, inter: str = "nn", cdl=cdl, no=no, order="t-f"
):
    ssim_scores = []
    for _ in tqdm(range(number_of_trials)):
        binary_values = binary_sources(
            [batch_size, num_tx, num_streams_per_tx, number_of_bits]
        )
        encoded_binary_values = ldpc_encoder(binary_values)
        qam_symbols = qam_mapper(encoded_binary_values)
        mapped_qam_symbol = resource_grid_mapper(qam_symbols)
        h_freq = sampling_channel_freq(cdl=cdl)

        # response signal on frequency domain via CDL channel model + AWGN noise
        response_symbols = response_freqency_domain(mapped_qam_symbol, h_freq, no=no)

        channel_estimator = ChannelEstimator(interpolation_factor=inter, order=order)
        h_est_hat, err_var = channel_estimator.estimate(response_symbols)

        h_est_hat = reshape_data(h_est_hat.numpy())

        h_est = remove_nulls_subcarriers(h_freq)
        h_est = reshape_data(h_est.numpy())

        ssim_score = ssim_func(
            torch.from_numpy(h_est_hat),
            torch.from_numpy(h_est),
        )
        ssim_scores.append(ssim_score)

    return np.mean(ssim_scores)

In [15]:
start = time.time()
ssim_score = ssim_channel_estimation_ai(1)
end = time.time()
print(f"Time taken for AI: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00, 20.96it/s]

Time taken for AI: 0.048670053482055664 seconds


In [16]:
start = time.time()
ssim_score = ssim_channel_estimation_ls(1, inter="nn")
end = time.time()
print(f"Time taken for LS+NN: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00, 17.32it/s]

Time taken for LS+NN: 0.05892300605773926 seconds


In [17]:
start = time.time()
ssim_score = ssim_channel_estimation_ls(1, inter="lin")
end = time.time()
print(f"Time taken for LS+Lin: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00, 17.31it/s]

Time taken for LS+Lin: 0.05888009071350098 seconds


In [18]:
start = time.time()
ssim_score = ssim_channel_estimation_ls(1, inter="lmmse")
end = time.time()
print(f"Time taken for LS+LMMSE: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00,  2.51it/s]

Time taken for LS+LMMSE: 0.3992776870727539 seconds


In [19]:
import time
import numpy as np

hyperparameters = {
    "ebno_dbs": np.arange(-20, 21, 5),
    "speeds": np.arange(0, 11, 5),
}

number_of_trials = 200
ssim_scores = {}
inference_times = {}  # Dictionary to store inference times

for speed in hyperparameters["speeds"]:
    cdl = CDL(
        cdl_model,
        delay_spread,
        carrier_frequency,
        ut_array,
        bs_array,
        direction,
        min_speed=speed,
    )
    for ebno_db in hyperparameters["ebno_dbs"]:
        ssim_score = {}
        inference_time = {}  # Store times for each model
        no = ebnodb2no(ebno_db, bits_per_symbol, code_rate, rg)

        # Measure inference time for VAE
        start_time = time.time()
        ssim_score["vae"] = ssim_channel_estimation_ai(number_of_trials, no=no, cdl=cdl)
        inference_time["vae"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+NN
        start_time = time.time()
        ssim_score["ls+nn"] = ssim_channel_estimation_ls(
            number_of_trials, inter="nn", cdl=cdl, no=no
        )
        inference_time["ls+nn"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LIN
        start_time = time.time()
        ssim_score["ls+lin"] = ssim_channel_estimation_ls(
            number_of_trials, inter="lin", cdl=cdl, no=no
        )
        inference_time["ls+lin"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        ssim_score["ls+lmmse: t-f"] = ssim_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="t-f"
        )
        inference_time["ls+lmmse: t-f"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        ssim_score["ls+lmmse: t-f-s"] = ssim_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="t-f-s"
        )
        inference_time["ls+lmmse: t-f-s"] = (
            time.time() - start_time
        ) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        ssim_score["ls+lmmse: s-t-f"] = ssim_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="s-t-f"
        )
        inference_time["ls+lmmse: s-t-f"] = (
            time.time() - start_time
        ) / number_of_trials

        ssim_scores[f"speed_{speed}_ebno_db_{ebno_db}"] = ssim_score
        inference_times[f"speed_{speed}_ebno_db_{ebno_db}"] = inference_time

100%|██████████| 200/200 [00:09<00:00, 20.94it/s]


In [20]:
import json
import os

save_dir = f"{root_dir}/results/eval/ssim/ssim.json"
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

# Key for the current data
key = f"txant_{num_ut_ant}_rxant_{num_bs_ant}"

# Read existing data if the file exists
existing_data = {}
if os.path.exists(save_dir):
    try:
        with open(save_dir, "r") as f:
            existing_data = json.load(f)
    except json.JSONDecodeError:
        # Handle empty or corrupted JSON file
        existing_data = {}

# Update or append the new data
existing_data[key] = ssim_scores

# Write back to the file
with open(save_dir, "w") as f:
    json.dump(existing_data, f, indent=4)

In [21]:
import json
import os

save_dir = f"{root_dir}/results/eval/ssim/time.json"
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

# Key for the current data
key = f"txant_{num_ut_ant}_rxant_{num_bs_ant}"

# Read existing data if the file exists
existing_data = {}
if os.path.exists(save_dir):
    try:
        with open(save_dir, "r") as f:
            existing_data = json.load(f)
    except json.JSONDecodeError:
        # Handle empty or corrupted JSON file
        existing_data = {}

# Update or append the new data
existing_data[key] = inference_times

# Write back to the file
with open(save_dir, "w") as f:
    json.dump(existing_data, f, indent=4)

# BER

In [22]:
def ber_channel_estimation_ai(number_of_trials, no=no, cdl=cdl):
    ber_scores = []
    for _ in tqdm(range(number_of_trials)):
        binary_values = binary_sources(
            [batch_size, num_tx, num_streams_per_tx, number_of_bits]
        )
        encoded_binary_values = ldpc_encoder(binary_values)
        qam_symbols = qam_mapper(encoded_binary_values)
        mapped_qam_symbol = resource_grid_mapper(qam_symbols)
        modulated_qam_symbols = ofdm_modulation(mapped_qam_symbol)
        h_time = sampling_channel_time(cdl=cdl)
        response_symbols = response_time_domain(modulated_qam_symbols, h_time, no=no)
        demodulated_response_symbols = ofdm_demodulation(response_symbols)

        # Channel estimation
        pilot_matrix = get_pilot_matrix(demodulated_response_symbols)
        # pilot_matrix = replicate_to_shape(pilot_matrix, h_freq_shape)
        pilot_matrix = reshape_data(pilot_matrix)
        pilot_matrix = scaler.fit_transform(pilot_matrix, False).to(device)
        h_est_hat, *values = vae(pilot_matrix)
        h_est_hat = scaler.inverse_transform(h_est_hat.cpu().detach())
        h_est_hat = reversed_reshape_data(h_est_hat)

        mapped_qam_symbol_hat, no_eff = lmmse_equalizer(
            demodulated_response_symbols, h_est_hat, 0
        )

        binary_values_hat = qam_demapper(mapped_qam_symbol_hat, bits_per_symbol, no_eff)

        decoded_binary_values_hat = ldpc_decoder(binary_values_hat)

        error_rate = ber(binary_values, decoded_binary_values_hat)
        ber_scores.append(error_rate)

    return np.mean(ber_scores)


def ber_channel_estimation_ls(
    number_of_trials, inter: str = "nn", cdl=cdl, no=no, order="t-f"
):
    for _ in tqdm(range(number_of_trials)):
        binary_values = binary_sources(
            [batch_size, num_tx, num_streams_per_tx, number_of_bits]
        )
        encoded_binary_values = ldpc_encoder(binary_values)
        qam_symbols = qam_mapper(encoded_binary_values)
        mapped_qam_symbol = resource_grid_mapper(qam_symbols)
        modulated_qam_symbols = ofdm_modulation(mapped_qam_symbol)
        h_time = sampling_channel_time(cdl=cdl)
        response_symbols = response_time_domain(modulated_qam_symbols, h_time, no=no)
        demodulated_response_symbols = ofdm_demodulation(response_symbols)

        channel_estimator = ChannelEstimator(interpolation_factor=inter, order=order)
        h_est_hat, err_var = channel_estimator.estimate(demodulated_response_symbols)
        mapped_qam_symbol_hat, no_eff = lmmse_equalizer(
            demodulated_response_symbols, h_est_hat, err_var
        )

        binary_values_hat = qam_demapper(mapped_qam_symbol_hat, bits_per_symbol, no_eff)

        decoded_binary_values_hat = ldpc_decoder(binary_values_hat)

        error_rate = ber(binary_values, decoded_binary_values_hat)

    return float(error_rate)

In [23]:
start = time.time()
ber_score = ber_channel_estimation_ai(1)
print("BER AI: ", ber_score)
end = time.time()
print(f"Time taken for AI: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00,  3.48it/s]

BER AI:  0.0
Time taken for AI: 0.2879765033721924 seconds


In [24]:
start = time.time()
ber_score = ber_channel_estimation_ls(1, inter="nn")
print("BER LS+NN: ", ber_score)
end = time.time()
print(f"Time taken for LS+NN: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00,  3.67it/s]

BER LS+NN:  0.0
Time taken for LS+NN: 0.2737462520599365 seconds


In [25]:
start = time.time()
ber_score = ber_channel_estimation_ls(1, inter="lmmse")
print("BER LS+LMMSE: ", ber_score)
end = time.time()
print(f"Time taken for LS+LMMSE: {end - start} seconds")

100%|██████████| 1/1 [00:00<00:00,  1.60it/s]

BER LS+LMMSE:  0.0
Time taken for LS+LMMSE: 0.6273691654205322 seconds


In [26]:
import time
import numpy as np

hyperparameters = {
    "ebno_dbs": np.arange(-20, 21, 5),
    "speeds": np.arange(0, 11, 5),
}

number_of_trials = 200
ber_scores = {}
inference_times = {}  # Dictionary to store inference times

for speed in hyperparameters["speeds"]:
    cdl = CDL(
        cdl_model,
        delay_spread,
        carrier_frequency,
        ut_array,
        bs_array,
        direction,
        min_speed=speed,
    )
    for ebno_db in hyperparameters["ebno_dbs"]:
        ber_score = {}
        inference_time = {}  # Store times for each model
        no = ebnodb2no(ebno_db, bits_per_symbol, code_rate, rg)

        # Measure inference time for VAE
        start_time = time.time()
        ber_score["vae"] = ber_channel_estimation_ai(number_of_trials, no=no, cdl=cdl)
        inference_time["vae"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+NN
        start_time = time.time()
        ber_score["ls+nn"] = ber_channel_estimation_ls(
            number_of_trials, inter="nn", cdl=cdl, no=no
        )
        inference_time["ls+nn"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LIN
        start_time = time.time()
        ber_score["ls+lin"] = ber_channel_estimation_ls(
            number_of_trials, inter="lin", cdl=cdl, no=no
        )
        inference_time["ls+lin"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        ber_score["ls+lmmse: t-f"] = ber_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="t-f"
        )
        inference_time["ls+lmmse: t-f"] = (time.time() - start_time) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        ber_score["ls+lmmse: t-f-s"] = ber_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="t-f-s"
        )
        inference_time["ls+lmmse: t-f-s"] = (
            time.time() - start_time
        ) / number_of_trials

        # Measure inference time for LS+LMMSE
        start_time = time.time()
        ber_score["ls+lmmse: s-t-f"] = ber_channel_estimation_ls(
            number_of_trials, inter="lmmse", cdl=cdl, no=no, order="s-t-f"
        )
        inference_time["ls+lmmse: s-t-f"] = (
            time.time() - start_time
        ) / number_of_trials

        ber_scores[f"speed_{speed}_ebno_db_{ebno_db}"] = ber_score
        inference_times[f"speed_{speed}_ebno_db_{ebno_db}"] = inference_time

100%|██████████| 200/200 [00:51<00:00,  3.86it/s]


In [27]:
import json
import os

save_dir = f"{root_dir}/results/eval/ber/ber.json"
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

# Key for the current data
key = f"txant_{num_ut_ant}_rxant_{num_bs_ant}"

# Read existing data if the file exists
existing_data = {}
if os.path.exists(save_dir):
    try:
        with open(save_dir, "r") as f:
            existing_data = json.load(f)
    except json.JSONDecodeError:
        # Handle empty or corrupted JSON file
        existing_data = {}

# Update or append the new data
existing_data[key] = ber_scores

# Write back to the file
with open(save_dir, "w") as f:
    json.dump(existing_data, f, indent=4)

In [28]:
import json
import os

save_dir = f"{root_dir}/results/eval/ber/time.json"
os.makedirs(os.path.dirname(save_dir), exist_ok=True)

# Key for the current data
key = f"txant_{num_ut_ant}_rxant_{num_bs_ant}"

# Read existing data if the file exists
existing_data = {}
if os.path.exists(save_dir):
    try:
        with open(save_dir, "r") as f:
            existing_data = json.load(f)
    except json.JSONDecodeError:
        # Handle empty or corrupted JSON file
        existing_data = {}

# Update or append the new data
existing_data[key] = inference_times

# Write back to the file
with open(save_dir, "w") as f:
    json.dump(existing_data, f, indent=4)